In [5]:
#Extract ALl Data from BSE

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import os
import time


def get_equity_data(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    dict1 = dict()

    div_elements = soup.find_all('div', class_='col-lg-13')
    for element in div_elements:
        try:
            tr_tags = element.find('tbody').find_all('tr')
        except:
            continue
        for tr_tag in tr_tags:
            try:
                key = tr_tag.find('td', class_='textsr').text.strip()
                value = tr_tag.find('td', class_='textvalue ng-binding').text.strip()
                
                # Assign the key-value pair
                dict1[key] = value
            except:
                continue

    # Convert the extracted dictionary to DataFrame
    row_list = []
    data = dict1
    row_list.append(data.values())
    df_equity = pd.DataFrame(row_list, columns=data.keys())

    # Check if 'PE/PB' column exists in the DataFrame
    if 'PE/PB' in df_equity.columns:
        # Split the 'PE/PB' column into two columns 'PE' and 'PB'
        df_equity[['PE', 'PB']] = df_equity['PE/PB'].str.split(' / ', expand=True)
        
        # Drop the original 'PE/PB' column
        df_equity.drop(columns=['PE/PB'], inplace=True)

    return df_equity


def get_corpgov_data(driver):
    try:
        WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.LINK_TEXT, "Corporate Governance"))
        )

        element = driver.find_element(By.LINK_TEXT, "Corporate Governance")
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        time.sleep(1)
        element.click()

        time.sleep(3)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        element = soup.find_all('table', class_='ng-scope')[4]
        row_list = []
        tr_body = element.find('tbody').find_all('tr')

        for tr_tag in tr_body:
            l = []
            try:
                td_tags = tr_tag.find_all('td', class_='ng-binding')
                for td_tag in td_tags:
                    l.append(td_tag.text.strip())
            except:
                continue
            row_list.append(l)

        # Original column data
        column_data = ['Sr', 'Title (Mr/Ms)', 'Name of the Director', 'DIN', 'Category', 
                       'Whether the director is disqualified?', 'Start Date of disqualification', 
                       'End Date of disqualification', 'Details of disqualification', 
                       'Current status', 'Whether special resolution passed?', 
                       'Date of passing special resolution', 'Initial Date of Appointment', 
                       'Date of Re-appointment', 'Date of cessation', 'Tenure of Director (in months)', 
                       'No of Directorship in listed entities', 
                       'No of Independent Directorship in listed entities', 
                       'Number of memberships in Audit/ Stakeholder Committee(s)', 
                       'No of post of Chairperson in Audit/ Stakeholder Committee', 
                       'Reason for Cessation', 'Notes for not providing PAN', 'Notes for not providing DIN']

        dataframe = pd.DataFrame(row_list, columns=column_data)

        # Combine 'Title (Mr/Ms)' and 'Name of the Director' into one column
        dataframe['Director Name'] = dataframe['Title (Mr/Ms)'] + " " + dataframe['Name of the Director']

        # Dropping original columns after combining
        dataframe.drop(columns=['Title (Mr/Ms)', 'Name of the Director'], inplace=True)

        # Create a filtered DataFrame with only the needed columns
        needed_row_list = []
        for lis in row_list:
            combined_name = lis[1] + " " + lis[2]  # Combining title and name for filtered DataFrame
            row_list1 = [lis[0], combined_name, lis[4], lis[9], lis[12], lis[13]]
            needed_row_list.append(row_list1)

        # Define the new column names for the filtered DataFrame
        column_data1 = ['Sr', 'Director Name', 'Category', 'Current status', 
                        'Initial Date of Appointment', 'Date of Re-appointment']
        
        dataframe1 = pd.DataFrame(needed_row_list, columns=column_data1)

    except Exception as e:
        print(f"Error in get_corpgov_data: {e}")
        dataframe = pd.DataFrame()
        dataframe1 = pd.DataFrame()

    return dataframe, dataframe1

    
def get_quarterly_data(driver):
    try:
        table_html = driver.find_element(By.CSS_SELECTOR, 'div.tab-pane.active.largetable').get_attribute('outerHTML')
        soup = BeautifulSoup(table_html, 'html.parser')
        headers = [header.text.strip() for header in soup.find_all('td', class_='tableheading')]
        
        rows = []
        for row in soup.find_all('tr'):
            cells = [cell.text.strip() for cell in row.find_all('td', class_='tdcolumn')]
            if cells:
                rows.append(cells)
        
        consistent_rows = [row for row in rows if len(row) == len(headers)]
        df_quarterly = pd.DataFrame(consistent_rows, columns=headers)
        
        # Transpose the dataframe to make it more manageable
        df_quarterly_transposed = df_quarterly.T
        new_header = df_quarterly_transposed.iloc[0]
        df_quarterly_transposed = df_quarterly_transposed[1:]
        df_quarterly_transposed.columns = new_header
        
        # Rename the 'Results (in Cr.)' column
        df_quarterly_transposed.rename(columns={'Results (in Cr.)  View in (Million)': 'Year'}, inplace=True)
        
        # Split the '52 W H/L' column into '52 W H' and '52 W L'
        if '52 W H/L' in df_quarterly_transposed.columns:
            df_quarterly_transposed[['52 W H', '52 W L']] = df_quarterly_transposed['52 W H/L'].str.split('/', expand=True)
            df_quarterly_transposed.drop(columns=['52 W H/L'], inplace=True)
        
        df_quarterly_transposed.reset_index(inplace=True)
        df_quarterly_transposed.rename(columns={'index': 'Peer Company'}, inplace=True)

        # Filter the dataframe to include only the required columns
        required_columns = [
            'Peer Company', 'LTP', 'Change %', 'Year', 'Sales', 'PAT', 'Equity', 
            'Face Value', 'OPM %', 'NPM %', 'EPS', 'CEPS', 'PE', 
            '52 W H', '52 W L'
        ]
        
        df_quarterly_filtered = df_quarterly_transposed[required_columns]

        return df_quarterly_filtered
    except Exception as e:
        print(f"Error in get_quarterly_data: {e}")
        return pd.DataFrame()


def get_annual_data(driver):
    try:
        annual_trends_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'aanualtrd')))
        driver.execute_script("arguments[0].scrollIntoView(true);", annual_trends_button)
        time.sleep(1)
        annual_trends_button.click()
        time.sleep(5)
        
        table_html = driver.find_element(By.CSS_SELECTOR, 'div.tab-pane.active.largetable').get_attribute('outerHTML')
        soup = BeautifulSoup(table_html, 'html.parser')
        headers = [header.text.strip() for header in soup.find_all('td', class_='tableheading')]
        
        rows = []
        for row in soup.find_all('tr'):
            cells = [cell.text.strip() for cell in row.find_all('td', class_='tdcolumn')]
            if cells:
                rows.append(cells)
        
        consistent_rows = [row for row in rows if len(row) == len(headers)]
        df_annual = pd.DataFrame(consistent_rows, columns=headers)
        
        # Transpose the dataframe to make it more manageable
        df_annual_transposed = df_annual.T
        new_header = df_annual_transposed.iloc[0]
        df_annual_transposed = df_annual_transposed[1:]
        df_annual_transposed.columns = new_header
        
        # Rename the 'Results (in Cr.)' column
        df_annual_transposed.rename(columns={'Results (in Cr.)  View in (Million)': 'Year'}, inplace=True)
        
        # Split the '52 W H/L' column into '52 W H' and '52 W L'
        if '52 W H/L' in df_annual_transposed.columns:
            df_annual_transposed[['52 W H', '52 W L']] = df_annual_transposed['52 W H/L'].str.split('/', expand=True)
            df_annual_transposed.drop(columns=['52 W H/L'], inplace=True)
        
        df_annual_transposed.reset_index(inplace=True)
        df_annual_transposed.rename(columns={'index': 'Peer Company'}, inplace=True)

        # Filter the dataframe to include only the required columns
        required_columns = [
            'Peer Company', 'LTP', 'Change %', 'Year', 'Sales', 'PAT', 'Equity', 
            'Face Value', 'OPM %', 'NPM %', 'EPS', 'CEPS', 'PE', 
            '52 W H', '52 W L'
        ]
        
        df_annual_filtered = df_annual_transposed[required_columns]

        return df_annual_filtered
    except Exception as e:
        print(f"Error in get_annual_data: {e}")
        return pd.DataFrame()

def main(url):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    service = Service()  # Add chromedriver path if needed
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(url)
        
        def click_element(driver, xpath):
            try:
                element = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, xpath))
                )
                driver.execute_script("arguments[0].scrollIntoView(true);", element)
                ActionChains(driver).move_to_element(element).click().perform()
            except Exception as e:
                print(f"Click failed: {e}")

        click_element(driver, "//div[@id='l14']")
        click_element(driver, "//a[contains(text(), 'Peer Group')]")
        
        time.sleep(5)
        
        df_quarterly = get_quarterly_data(driver)
        df_annual = get_annual_data(driver)
        # Extract company name from URL
        l = url.split('/')
        company_name = l[4].strip().capitalize() #Pass the company name here
        df_corpgov, df_corpgov_filtered = get_corpgov_data(driver)
        df_equity = get_equity_data(driver)

        # Add company name and industry to the DataFrames
        for df in [df_equity, df_corpgov, df_corpgov_filtered, df_quarterly, df_annual]:
            df['Company Name'] = company_name
            df['Industry Name'] = "Auto Components & Equipments"

        # Define the Excel file path
        excel_path = "Combined_data_BSE.xlsx"

        # Check if the file exists
        if os.path.exists(excel_path):
            # Load existing data
            with pd.ExcelFile(excel_path) as xls:
                df_quarterly_existing = pd.read_excel(xls, sheet_name='Peer_Quarterly Data')
                df_annual_existing = pd.read_excel(xls, sheet_name='Peer_Annual Data')
                df_corpgov_existing = pd.read_excel(xls, sheet_name='Corporate Governance')
                df_corpgov_filtered_existing = pd.read_excel(xls, sheet_name='Board_Members')
                df_equity_existing = pd.read_excel(xls, sheet_name='Equity Data')

            # Append new data to existing data
            df_quarterly = pd.concat([df_quarterly_existing, df_quarterly], ignore_index=True)
            df_annual = pd.concat([df_annual_existing, df_annual], ignore_index=True)
            df_corpgov = pd.concat([df_corpgov_existing, df_corpgov], ignore_index=True)
            df_corpgov_filtered = pd.concat([df_corpgov_filtered_existing, df_corpgov_filtered], ignore_index=True)
            df_equity = pd.concat([df_equity_existing, df_equity], ignore_index=True)

        # Save all DataFrames to different sheets in an Excel file
        with pd.ExcelWriter(excel_path, engine='openpyxl', mode='w') as writer:
            df_quarterly.to_excel(writer, sheet_name='Peer_Quarterly Data', index=False)
            df_annual.to_excel(writer, sheet_name='Peer_Annual Data', index=False)
            df_corpgov.to_excel(writer, sheet_name='Corporate Governance', index=False)
            df_corpgov_filtered.to_excel(writer, sheet_name='Board_Members', index=False)
            df_equity.to_excel(writer, sheet_name='Equity Data', index=False)

    finally:
        driver.quit()

if __name__ == "__main__":
    # Update this URL with the one you want to scrape
    urls = [
        #'https://www.bseindia.com/stock-share-price/shivam-autotech-ltd/shivamauto/532776/',
       #'https://www.bseindia.com/stock-share-price/fiem-industries-ltd/fiemind/532768/',
          #'https://www.bseindia.com/stock-share-price/india-nippon-electricals-ltd/indnippon/532240/',
        #'https://www.bseindia.com/stock-share-price/lumax-auto-technologies-ltd/lumaxtech/532796/',
           #'https://www.bseindia.com/stock-share-price/sandhar-technologies-ltd/sandhar/541163/',
         #'https://www.bseindia.com/stock-share-price/nrb-bearings-ltd/nrbbearing/530367/',
        #'https://www.bseindia.com/stock-share-price/mmforgings-ltd/mmfl/522241/',
        #'https://www.bseindia.com/stock-share-price/steel-strips-wheels-ltd/sswl/513262/',
        #'https://www.bseindia.com/stock-share-price/igarashi-motors-india-ltd/igarashi/517380/',
        #'https://www.bseindia.com/stock-share-price/jay-bharat-maruti-ltd/jaybarmaru/520066/',
        #'https://www.bseindia.com/stock-share-price/g-n-a-axles-ltd/gna/540124/',
       'https://www.bseindia.com/stock-share-price/talbros-automotive-components-ltd/talbroauto/505160/',
       # 'https://www.bseindia.com/stock-share-price/lumax-industries-ltd/lumaxind/517206/',
       # 'https://www.bseindia.com/stock-share-price/wheels-india-ltd/wheels/590073/',
       # 'https://www.bseindia.com/stock-share-price/bharat-seats-ltd/bharatse/523229/',
        #'https://www.bseindia.com/stock-share-price/alicon-castalloy-limited/alicon/531147/',
        
        
    ]

    for url in urls:
        main(url)


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

def get_financial_results(driver, company, industry, excel_path):
    try:
        # Expand the financial results section
        element_to_click = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a#afi[data-toggle='collapse'][data-parent='#accordion']"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", element_to_click)
        driver.execute_script("arguments[0].click();", element_to_click)

        time.sleep(3)

        # Click on Quarterly Trends
        quarterly_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a#l61"))
        )
        driver.execute_script("arguments[0].click();", quarterly_element)

        time.sleep(6)

        # Parse the page content with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tables = soup.find_all('table', class_="ng-binding")

        if not tables:
            print("No tables found with class 'ng-binding'")
            return pd.DataFrame(), pd.DataFrame()

        # Process Quarterly Trends
        financial_quarterly_df = process_table(tables[0], 'Quarterly')
        financial_quarterly_df['Company Name'] = company
        financial_quarterly_df['Industry Name'] = industry

        # Click on Annual Trends
        annual_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.LINK_TEXT, "Annual Trends"))
        )
        driver.execute_script("arguments[0].click();", annual_element)

        time.sleep(3)

        # Parse Annual Trends data
        soup1 = BeautifulSoup(driver.page_source, 'html.parser')
        tables1 = soup1.find_all('table', class_="ng-binding")

        if len(tables1) < 4:
            print(f"Expected at least 4 tables, but found {len(tables1)}")
            return pd.DataFrame(), pd.DataFrame()

        # Process Annual Trends
        financial_annual_df = process_table(tables1[3], 'Annual')
        financial_annual_df['Company Name'] = company
        financial_annual_df['Industry Name'] = industry

        return financial_quarterly_df, financial_annual_df

    except Exception as e:
        print(f"An error occurred for {company}: {str(e)}")
        return pd.DataFrame(), pd.DataFrame()

def process_table(table, trend_type):
    row_list = []
    columns = []
    
    try:
        tr_heads = table.find('thead').find_all('tr')
        tr_body = table.find('tbody').find_all('tr')
    except AttributeError:
        print(f"Table structure for {trend_type} Trends is not as expected")
        return pd.DataFrame(), pd.DataFrame()
    
    # Extract data from body rows
    for tr_tag in tr_body:
        l = []
        try:
            td_tags = tr_tag.find_all('td', class_='tdcolumn')
            for td_tag in td_tags:
                l.append(td_tag.text.strip())
        except:
            continue
        row_list.append(l)
    
    # Extract column headers
    for th_tag in tr_heads:
        td_tags = th_tag.find_all('td', class_='tableheading')
        for td_tag in td_tags:
            columns.append(td_tag.text.strip())
    
    dataframe = pd.DataFrame(row_list, columns=columns)
    dataframe = dataframe[:-3]  # Remove unwanted rows
    dataframe = dataframe.drop(dataframe.index[0])  # Remove header row
    if trend_type == 'Quarterly':
        if dataframe.columns[-1].startswith('FY'):
            dataframe = dataframe.iloc[:, :-1]  # Drop the last column
    
    # Transpose the data
    dataframe = dataframe.set_index(dataframe.columns[0]).transpose()
    dataframe.reset_index(inplace=True)
    dataframe.rename(columns={'index': 'Date' if trend_type == 'Quarterly' else 'Year'}, inplace=True)
    
    return dataframe

def scrape_meetings(driver, url, industry_name):
    try:
        # Extract company name from URL
        l = url.split('/')
        company_name = l[4].strip().capitalize()

        # Open the main page
        driver.get(url)

        # Expand the meetings section
        element_to_click = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a#ame[data-toggle='collapse'][data-parent='#accordion']"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", element_to_click)
        driver.execute_script("arguments[0].click();", element_to_click)

        time.sleep(3)  # Wait for the section to expand

        # Function to extract table data for meetings
        def extract_meeting_data(link_id, state_attr, meeting_type):
            meetings_link = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, link_id))
            )
            driver.execute_script("arguments[0].click();", meetings_link)

            time.sleep(6)  # Wait for the page to load

            # Parse the page content with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Locate the table for the meeting data
            table = soup.find('table', {'ng-if': f"loader.{state_attr}=='loaded'"})
            data = []

            if table:
                rows = table.find_all('tr')[1:]  # Skip the header row
                for row in rows:
                    cols = row.find_all('td')
                    if len(cols) >= 2:  # Ensure there are at least 2 columns
                        meeting_date = cols[0].text.strip()
                        purpose = cols[1].text.strip()
                        data.append([meeting_date, purpose, meeting_type, company_name, industry_name])
            else:
                print(f"Table for {state_attr} not found.")

            # Navigate back to the main page
            driver.back()
            time.sleep(3)  # Wait for the page to load again

            return data

        # Step 1: Scrape Board Meetings Data
        board_meetings_data = extract_meeting_data("l71", "BMState", "Board Meeting")
        board_meetings_df = pd.DataFrame(board_meetings_data, columns=['Meeting Date', 'Purpose', 'Meeting Type', 'Company Name', 'Industry'])

        # Step 2: Scroll again to find and click the Shareholder Meetings link
        element_to_click = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a#ame[data-toggle='collapse'][data-parent='#accordion']"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", element_to_click)
        driver.execute_script("arguments[0].click();", element_to_click)

        time.sleep(3)  # Wait for the section to expand again

        # Step 3: Scrape Shareholder Meetings Data
        shareholder_meetings_data = extract_meeting_data("l72", "SHState", "Shareholder Meeting")
        shareholder_meetings_df = pd.DataFrame(shareholder_meetings_data, columns=['Meeting Date', 'Purpose', 'Meeting Type', 'Company Name', 'Industry'])

        # Combine both dataframes into one
        combined_meetings_df = pd.concat([board_meetings_df, shareholder_meetings_df], ignore_index=True)

        return combined_meetings_df

    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on error

def main():
    # Configure Selenium WebDriver
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run browser in headless mode
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    service = Service()  # Add chromedriver path if needed
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # List of URLs to scrape
    urls = [
        #'https://www.bseindia.com/stock-share-price/shivam-autotech-ltd/shivamauto/532776/',
        #'https://www.bseindia.com/stock-share-price/fiem-industries-ltd/fiemind/532768/',
        #'https://www.bseindia.com/stock-share-price/india-nippon-electricals-ltd/indnippon/532240/',
        #'https://www.bseindia.com/stock-share-price/lumax-auto-technologies-ltd/lumaxtech/532796/',
        #'https://www.bseindia.com/stock-share-price/sandhar-technologies-ltd/sandhar/541163/',
         #'https://www.bseindia.com/stock-share-price/nrb-bearings-ltd/nrbbearing/530367/',
        #'https://www.bseindia.com/stock-share-price/mmforgings-ltd/mmfl/522241/',
        #'https://www.bseindia.com/stock-share-price/steel-strips-wheels-ltd/sswl/513262/',
        #'https://www.bseindia.com/stock-share-price/igarashi-motors-india-ltd/igarashi/517380/',
        #'https://www.bseindia.com/stock-share-price/jay-bharat-maruti-ltd/jaybarmaru/520066/',
        #'https://www.bseindia.com/stock-share-price/g-n-a-axles-ltd/gna/540124/',
        'https://www.bseindia.com/stock-share-price/talbros-automotive-components-ltd/talbroauto/505160/',
        #'https://www.bseindia.com/stock-share-price/lumax-industries-ltd/lumaxind/517206/',
        #'https://www.bseindia.com/stock-share-price/wheels-india-ltd/wheels/590073/',
        #'https://www.bseindia.com/stock-share-price/bharat-seats-ltd/bharatse/523229/',
        #'https://www.bseindia.com/stock-share-price/alicon-castalloy-limited/alicon/531147/',
    ]

    # Specify the path where you want to save the Excel file
    excel_path = "Financial_Bse_data.xlsx"

    all_financial_quarterly_data = pd.DataFrame()  # Changed name
    all_financial_annual_data = pd.DataFrame()      # Changed name
    all_meetings_data = pd.DataFrame()              # Unchanged

    for link in urls:
        driver.get(link)
        time.sleep(2)

        # Extract company name from URL
        l = link.split('/')
        company = l[4].strip().capitalize()

        # Get financial results with industry name
        financial_quarterly_data, financial_annual_data = get_financial_results(driver, company, "Auto Components & Equipments", excel_path)

        all_financial_quarterly_data = pd.concat([all_financial_quarterly_data, financial_quarterly_data], ignore_index=True)
        all_financial_annual_data = pd.concat([all_financial_annual_data, financial_annual_data], ignore_index=True)

        # Get meeting data
        meetings_data = scrape_meetings(driver, link, "Auto Components & Equipments")
        all_meetings_data = pd.concat([all_meetings_data, meetings_data], ignore_index=True)

    # Check if the Excel file exists
    if os.path.exists(excel_path):
        # Load existing data from Excel
        with pd.ExcelFile(excel_path) as xls:
            # Read existing data from sheets
            existing_quarterly_df = pd.read_excel(xls, sheet_name='Financial_Quarterly Trend')
            existing_annual_df = pd.read_excel(xls, sheet_name='Financial_Annual Trend')
            existing_meetings_df = pd.read_excel(xls, sheet_name='Meeting_Data')

            # Append new data to existing data
            all_financial_quarterly_data = pd.concat([existing_quarterly_df, all_financial_quarterly_data], ignore_index=True)
            all_financial_annual_data = pd.concat([existing_annual_df, all_financial_annual_data], ignore_index=True)
            all_meetings_data = pd.concat([existing_meetings_df, all_meetings_data], ignore_index=True)

    # Save the collected data to Excel
    with pd.ExcelWriter(excel_path) as writer:
        all_financial_quarterly_data.to_excel(writer, sheet_name='Financial_Quarterly Trend', index=False)  # Changed sheet name
        all_financial_annual_data.to_excel(writer, sheet_name='Financial_Annual Trend', index=False)      # Changed sheet name
        all_meetings_data.to_excel(writer, sheet_name='Meeting_Data', index=False)

    driver.quit()

if __name__ == "__main__":
    main()
